In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0,'D:/OneDrive/Major Project/Hybrid Model/utils')
import utils
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('D:/OneDrive/Major Project/Hybrid Model/GZ_2_Processed_classes.csv')
df

In [ ]:
arr = [1.4274031e-02, 9.7901762e-01, 4.1143298e-03, 8.9326799e-03,
       9.6137309e-01, 4.2684698e-01, 5.5259460e-01, 9.4123375e-01,
       2.4804175e-02, 1.0193929e-01, 5.7487273e-01, 2.0058504e-01,
       4.3642491e-02, 2.0016301e-01, 8.0416125e-01, 1.4221370e-03,
       1.3003081e-02, 7.1957707e-04, 1.1768907e-02, 7.1015954e-03,
       6.7494422e-02, 6.7107379e-02, 3.3029795e-02, 4.0997922e-02,
       2.0468235e-04, 8.9886487e-03, 8.7186694e-04, 2.4443269e-03,
       1.0589573e-01, 4.2550060e-01, 3.5367763e-01, 2.2649884e-02,
       8.3484709e-01, 4.5983166e-02, 4.8273802e-03, 1.2008548e-03,
       2.3412824e-02]
utils.classify(arr)

In [ ]:
galaxyClasses = []
for i in range(len(df)):  
    gClass = utils.classify(list(df.iloc[i, 1:]))
    galaxyClasses.append(gClass)
    print(i, " - ", gClass)

In [ ]:
df['Classes'] =  galaxyClasses
plt.figure(figsize = (15, 15))
sns.countplot(df.iloc[:, -1])
plt.show()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = ct.fit_transform(df.iloc[:, -1:].values)
X = pd.DataFrame.sparse.from_spmatrix(X)

In [ ]:
X

In [ ]:
columnNames = {}
classes = ['E0', 'E1', 'E2', 'E3', 'E4', 'E5', 'E7', 'Irregular', 'S0', 'SBx', 'Sx']
for i in range(11):
  columnNames[i] = classes[i]
X.rename(columns = columnNames, inplace=True)
X

In [ ]:
df = pd.concat([df, X], axis=1, join='inner')
df

In [ ]:
df.to_csv('../GZ_2_Simplified_classes.csv')

In [ ]:
classes = list(range(11))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import io
from PIL import Image
import matplotlib.pyplot as plt
from skimage.transform import resize
from tensorflow import keras
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from keras.applications.inception_resnet_v2 import InceptionResNetV2 as  PretrainedModel,preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

In [ ]:
import os
import re
import sys
import time
import numpy as np
from typing import Any, List, Tuple, Union
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, \
  LearningRateScheduler, ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pickle

In [ ]:
def generate_output_dir(outdir, run_desc):
    prev_run_dirs = []
    if os.path.isdir(outdir):
        prev_run_dirs = [x for x in os.listdir(outdir) if os.path.isdir(\
            os.path.join(outdir, x))]
    prev_run_ids = [re.match(r'^\d+', x) for x in prev_run_dirs]
    prev_run_ids = [int(x.group()) for x in prev_run_ids if x is not None]
    cur_run_id = max(prev_run_ids, default=-1) + 1
    run_dir = os.path.join(outdir, f'{cur_run_id:05d}-{run_desc}')
    assert not os.path.exists(run_dir)
    os.makedirs(run_dir)
    return run_dir

# From StyleGAN2
class Logger(object):
    """Redirect stderr to stdout, optionally print stdout to a file, and 
    optionally force flushing on both stdout and the file."""

    def __init__(self, file_name: str = None, file_mode: str = "w", \
                 should_flush: bool = True):
        self.file = None

        if file_name is not None:
            self.file = open(file_name, file_mode)

        self.should_flush = should_flush
        self.stdout = sys.stdout
        self.stderr = sys.stderr

        sys.stdout = self
        sys.stderr = self

    def __enter__(self) -> "Logger":
        return self

    def __exit__(self, exc_type: Any, exc_value: Any, \
                 traceback: Any) -> None:
        self.close()

    def write(self, text: str) -> None:
        """Write text to stdout (and a file) and optionally flush."""
        if len(text) == 0: 
            return

        if self.file is not None:
            self.file.write(text)

        self.stdout.write(text)

        if self.should_flush:
            self.flush()

    def flush(self) -> None:
        """Flush written text to both stdout and a file, if open."""
        if self.file is not None:
            self.file.flush()

        self.stdout.flush()

    def close(self) -> None:
        """Flush, close possible files, and remove 
            stdout/stderr mirroring."""
        self.flush()

        # if using multiple loggers, prevent closing in wrong order
        if sys.stdout is self:
            sys.stdout = self.stdout
        if sys.stderr is self:
            sys.stderr = self.stderr

        if self.file is not None:
            self.file.close()


In [ ]:
outdir = "D:/OneDrive/Major Project/Hybrid Model/params"
run_desc = "test-train"
num_classes = 11

run_dir = generate_output_dir(outdir, run_desc)
print(f"Results saved to: {run_dir}")

In [ ]:
class MyModelCheckpoint(ModelCheckpoint):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)

  def on_epoch_end(self, epoch, logs):
    super().on_epoch_end(epoch,logs)\

    # Also save the optimizer state
    filepath = self._get_file_path(epoch, logs=logs, batch=2)
    filepath = filepath.rsplit( ".", 1 )[ 0 ] 
    filepath += ".pkl"

    with open(filepath, 'wb') as fp:
      pickle.dump(
        {
          'opt': hybridModel.optimizer.get_config(),
          'epoch': epoch+1
         # Add additional keys if you need to store more values
        }, fp, protocol=pickle.HIGHEST_PROTOCOL)
    print('\nEpoch %05d: saving optimizer to %s' % (epoch + 1, filepath))

In [ ]:
def step_decay_schedule(initial_lr=1e-3, decay_factor=0.75, step_size=10):
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch/step_size))
    return LearningRateScheduler(schedule)

In [ ]:
def append_ext(fn):
    '''
    This function is used to take the GalaxyID from the CSV and append .jpg to it in order to denote the image names. 
    '''
    return fn + ".jpg"

df["id"] = df['GalaxyID'].astype(str).apply(append_ext) 

In [ ]:
datagenerator = ImageDataGenerator(
    fill_mode='nearest',
    cval=0,
    rescale=1/255,
    rotation_range=25,
    shear_range=0.2,
    width_shift_range=[0.1, 0.15],
    height_shift_range=[0.1, 0.15],
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=[0.4, 0.7],
    validation_split=0.025)

In [ ]:
classes=['E0', 'E1', 'E2', 'E3', 'E4', 'E5', 'E7', 'Irregular', 'S0', 'SBx', 'Sx']

train_generator = datagenerator.flow_from_dataframe(
    dataframe=df,
    directory="D:/Rahul Noronha/Shared Folder/Eighth Semester/Major Project/Data/images",
    x_col="id",
    y_col=classes,
    subset="training",
    batch_size=64,
    seed=123,
    shuffle=True,
    class_mode="raw",
    target_size=(299, 299))

validation_generator = datagenerator.flow_from_dataframe(
    dataframe=df,
    directory="D:/Rahul Noronha/Shared Folder/Eighth Semester/Major Project/Data/images",
    x_col="id",
    y_col=classes,
    subset="validation",
    batch_size=16,
    seed=123,
    shuffle=True,
    class_mode="raw",
    target_size=(299, 299))

STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = validation_generator.n // validation_generator.batch_size

In [ ]:

img_shape = (299, 299, 3)
num_classes = len(classes)

def build_model(img_shape, num_classes):
    hybridModel = Sequential()

    pretrained_model = PretrainedModel(
            input_shape = img_shape,
            weights = 'imagenet',
            include_top = False

    )
    for layer in pretrained_model.layers:
            layer.trainable=False

    hybridModel.add(pretrained_model)
    hybridModel.add(Flatten())
    hybridModel.add(Dense(11, activation='softmax'))
    optimizer = keras.optimizers.Adam()
    hybridModel.compile(optimizer, loss='mse', metrics=["accuracy"])
    return hybridModel

def train_model(hybridModel, initial_epoch=0, max_epochs=10):
    start_time = time.time()

    checkpoint_cb = MyModelCheckpoint(
        os.path.join(run_dir, 'model-{epoch:02d}-{val_loss:.2f}.hdf5'),
        monitor='val_loss',verbose=1)

    lr_sched_cb = step_decay_schedule(initial_lr=1.3379e-07, decay_factor=0.75, \
                                      step_size=5)
    cb = [checkpoint_cb, lr_sched_cb]

    hist = hybridModel.fit(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=validation_generator,
    validation_steps=STEP_SIZE_VALID,
    epochs=max_epochs,
    initial_epoch = initial_epoch,
    callbacks=cb)



In [ ]:
# with Logger(os.path.join(run_dir, 'log.txt')):
#   hybridModel = build_model(img_shape, num_classes)
#   train_model(hybridModel)

In [ ]:
MODEL_PATH = 'D:/OneDrive/Major Project/Hybrid Model/params/00008-test-train/model-39-0.06.hdf5'
OPT_PATH = 'D:/OneDrive/Major Project/Hybrid Model/params/00008-test-train/model-39-0.06.pkl'

In [ ]:
def load_model_data(model_path, opt_path):
    model = load_model(model_path)
    with open(opt_path, 'rb') as fp:
      d = pickle.load(fp)
      epoch = d['epoch']
      opt = d['opt']
      return epoch, model, opt

epoch, hybridModel, opt = load_model_data(MODEL_PATH, OPT_PATH)
hybridModel.compile(optimizer=tf.keras.optimizers.Adam.from_config(opt), loss='mse', metrics=["accuracy"])
with Logger(os.path.join(run_dir, 'log.txt')):
  train_model(hybridModel, initial_epoch=epoch, max_epochs=100)